# Vessel Emissions API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/VesselEmissionsAPI/VesselEmissionsAPI.ipynb) 

#### In this notebook, we will showcase the Vessel Emissions API and supported methods with examples.

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
!pip install signal-ocean

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

## Call the Vessel Emissions API
The Vessel Emissions API retrieves information about voyage emissions, vessel emissions and metrics for a single vessel or a vessel class.

In [2]:
from signal_ocean import Connection
from signal_ocean.vessel_emissions import VesselEmissionsAPI

connection = Connection(signal_ocean_api_key)
api = VesselEmissionsAPI(connection)

## Examples

### Get Emissions for a voyage

In [3]:
voyage_emissions = api.get_emissions_by_imo_and_voyage_number(imo=9412036, voyage_number=140)
voyage_emissions

{'ID': 'I8F9DC4VED6E22900',
 'IMO': 9412036,
 'VesselName': 'Sea Hymn',
 'VoyageNumber': 140,
 'VesselTypeID': 1,
 'VesselClassID': 86,
 'StartDate': '2020-09-19T23:58:04',
 'EndDate': '2020-11-06T11:58:50',
 'Deadweight': 116337,
 'Emissions': {'Voyage': {'CO2InTons': 2876.4055937543308,
   'COInTons': 2.513077972515762,
   'Ch4InTons': 0.05443490193174936,
   'N2OInTons': 0.14195200485341702,
   'NMVOCInTons': 2.7943249658298,
   'NOxInTons': 77.41626781768913,
   'SOxInTons': 1.773724989511432,
   'PmInTons': 2.066464647910779},
  'Ballast': {'CO2InTons': 264.4494860706905,
   'COInTons': 0.22848567573793283,
   'Ch4InTons': 0.004949148210929954,
   'N2OInTons': 0.01237287052732488,
   'NMVOCInTons': 0.2540562748277376,
   'NOxInTons': 6.759711598095162,
   'SOxInTons': 0.16126469502101193,
   'PmInTons': 0.07745391720997516},
  'Laden': {'CO2InTons': 1930.4620199910476,
   'COInTons': 1.694617755342644,
   'Ch4InTons': 0.03670652177637496,
   'N2OInTons': 0.09738671812238843,
   'N

### Include consumptions, distances, durations, speed_statistics in the response

In [4]:
voyage_emissions = api.get_emissions_by_imo_and_voyage_number(imo=9412036, voyage_number=140, 
                                                              include_consumptions=True,
                                                              include_distances=True,
                                                              include_durations=True,
                                                              include_speed_statistics=True
                                                             )
voyage_emissions

{'ID': 'I8F9DC4VED6E22900',
 'IMO': 9412036,
 'VesselName': 'Sea Hymn',
 'VoyageNumber': 140,
 'VesselTypeID': 1,
 'VesselClassID': 86,
 'StartDate': '2020-09-19T23:58:04',
 'EndDate': '2020-11-06T11:58:50',
 'Deadweight': 116337,
 'Emissions': {'Voyage': {'CO2InTons': 2876.4055937543308,
   'COInTons': 2.513077972515762,
   'Ch4InTons': 0.05443490193174936,
   'N2OInTons': 0.14195200485341702,
   'NMVOCInTons': 2.7943249658298,
   'NOxInTons': 77.41626781768913,
   'SOxInTons': 1.773724989511432,
   'PmInTons': 2.066464647910779},
  'Ballast': {'CO2InTons': 264.4494860706905,
   'COInTons': 0.22848567573793283,
   'Ch4InTons': 0.004949148210929954,
   'N2OInTons': 0.01237287052732488,
   'NMVOCInTons': 0.2540562748277376,
   'NOxInTons': 6.759711598095162,
   'SOxInTons': 0.16126469502101193,
   'PmInTons': 0.07745391720997516},
  'Laden': {'CO2InTons': 1930.4620199910476,
   'COInTons': 1.694617755342644,
   'Ch4InTons': 0.03670652177637496,
   'N2OInTons': 0.09738671812238843,
   'N

### Include efficiency metrics in the response

In [5]:
voyage_emissions = api.get_emissions_by_imo_and_voyage_number(imo=9412036, voyage_number=140, 
                                                              include_efficiency_metrics=True
                                                             )
voyage_emissions

{'ID': 'I8F9DC4VED6E22900',
 'IMO': 9412036,
 'VesselName': 'Sea Hymn',
 'VoyageNumber': 140,
 'VesselTypeID': 1,
 'VesselClassID': 86,
 'StartDate': '2020-09-19T23:58:04',
 'EndDate': '2020-11-06T11:58:50',
 'Deadweight': 116337,
 'Emissions': {'Voyage': {'CO2InTons': 2876.4055937543308,
   'COInTons': 2.513077972515762,
   'Ch4InTons': 0.05443490193174936,
   'N2OInTons': 0.14195200485341702,
   'NMVOCInTons': 2.7943249658298,
   'NOxInTons': 77.41626781768913,
   'SOxInTons': 1.773724989511432,
   'PmInTons': 2.066464647910779},
  'Ballast': {'CO2InTons': 264.4494860706905,
   'COInTons': 0.22848567573793283,
   'Ch4InTons': 0.004949148210929954,
   'N2OInTons': 0.01237287052732488,
   'NMVOCInTons': 0.2540562748277376,
   'NOxInTons': 6.759711598095162,
   'SOxInTons': 0.16126469502101193,
   'PmInTons': 0.07745391720997516},
  'Laden': {'CO2InTons': 1930.4620199910476,
   'COInTons': 1.694617755342644,
   'Ch4InTons': 0.03670652177637496,
   'N2OInTons': 0.09738671812238843,
   'N

### Include EU regulated emissions in the response

In [6]:
voyage_emissions = api.get_emissions_by_imo_and_voyage_number(imo=9412036, voyage_number=140, 
                                                              include_eu_emissions=True
                                                             )
voyage_emissions

{'ID': 'I8F9DC4VED6E22900',
 'IMO': 9412036,
 'VesselName': 'Sea Hymn',
 'VoyageNumber': 140,
 'VesselTypeID': 1,
 'VesselClassID': 86,
 'StartDate': '2020-09-19T23:58:04',
 'EndDate': '2020-11-06T11:58:50',
 'Deadweight': 116337,
 'Emissions': {'Voyage': {'CO2InTons': 2876.4055937543308,
   'COInTons': 2.513077972515762,
   'Ch4InTons': 0.05443490193174936,
   'N2OInTons': 0.14195200485341702,
   'NMVOCInTons': 2.7943249658298,
   'NOxInTons': 77.41626781768913,
   'SOxInTons': 1.773724989511432,
   'PmInTons': 2.066464647910779},
  'Ballast': {'CO2InTons': 264.4494860706905,
   'COInTons': 0.22848567573793283,
   'Ch4InTons': 0.004949148210929954,
   'N2OInTons': 0.01237287052732488,
   'NMVOCInTons': 0.2540562748277376,
   'NOxInTons': 6.759711598095162,
   'SOxInTons': 0.16126469502101193,
   'PmInTons': 0.07745391720997516},
  'Laden': {'CO2InTons': 1930.4620199910476,
   'COInTons': 1.694617755342644,
   'Ch4InTons': 0.03670652177637496,
   'N2OInTons': 0.09738671812238843,
   'N

### Get Emissions for a vessel

In [7]:
vessel_emissions = api.get_emissions_by_imo(imo=9412036, include_distances=True, include_eu_emissions=True)
# vessel_emissions

### Get Emissions for a vessel class

In [8]:
vessel_class_emissions = api.get_emissions_by_vessel_class_id(vessel_class_id=86)
# vessel_class_emissions

### Get Emissions for a vessel class using next page token

In [9]:
vessel_class_emissions_with_token = api.get_emissions_by_vessel_class_id(
    vessel_class_id=86, 
    token="eyJ2ZXNzZWxfY2xhc3NfaWQiOiA4NiwgImxhc3RfaWQiOiAiNjI1ZjUyZTlkMzMwZDA2MTBjOWFkZTEwIn0=",
    include_eu_emissions=True
)
# vessel_class_emissions_with_token

### Get Metrics for a vessel

In [11]:
vessel_metrics = api.get_metrics_by_imo(imo=9412036)
vessel_metrics

[{'IMO': 9412036,
  'Year': 2018,
  'VesselType': 'Tanker',
  'VesselTypeID': 1,
  'VesselClass': 'Aframax',
  'VesselClassID': 86,
  'Eexi': {'Value': 5.515052548670922,
   'Unit': 'g-CO2/ton mile',
   'Required': 3.288153751950862},
  'Eiv': {'Value': 4.69363738382239, 'Unit': 'g-CO2/ton mile'},
  'Aer': {'Value': 4.247291695332408,
   'Unit': 'g-CO2/dwt mile',
   'PoseidonPrinciplesClass': 'Oil Tanker 80000-119999 DWT',
   'PoseidonPrinciplesAlignmentInPercentage': 8.073580033903513,
   'PoseidonPrinciplesYearTarget': 3.93},
  'Cii': {'Value': 4.247291695332408,
   'Unit': 'g-CO2/capacity mile',
   'Rating': 'C',
   'Target': 4.264058648014019,
   'TargetYear': 2018}},
 {'IMO': 9412036,
  'Year': 2019,
  'VesselType': 'Tanker',
  'VesselTypeID': 1,
  'VesselClass': 'Aframax',
  'VesselClassID': 86,
  'Eexi': {'Value': 5.515052548670922,
   'Unit': 'g-CO2/ton mile',
   'Required': 3.288153751950862},
  'Eiv': {'Value': 4.69363738382239, 'Unit': 'g-CO2/ton mile'},
  'Aer': {'Value': 3

### Get Metrics for a vessel for a specific year

In [12]:
vessel_metrics = api.get_metrics_by_imo(imo=9412036, year=2020)
vessel_metrics

[{'IMO': 9412036,
  'Year': 2020,
  'VesselType': 'Tanker',
  'VesselTypeID': 1,
  'VesselClass': 'Aframax',
  'VesselClassID': 86,
  'Eexi': {'Value': 5.515052548670922,
   'Unit': 'g-CO2/ton mile',
   'Required': 3.288153751950862},
  'Eiv': {'Value': 4.69363738382239, 'Unit': 'g-CO2/ton mile'},
  'Aer': {'Value': 3.6762797538353507,
   'Unit': 'g-CO2/dwt mile',
   'PoseidonPrinciplesClass': 'Oil Tanker 80000-119999 DWT',
   'PoseidonPrinciplesAlignmentInPercentage': -1.7037498974505227,
   'PoseidonPrinciplesYearTarget': 3.74},
  'Cii': {'Value': 3.6762797538353507,
   'Unit': 'g-CO2/capacity mile',
   'Rating': 'B',
   'Target': 4.221418061533878,
   'TargetYear': 2020}}]

### Get Metrics for a vessel class

In [13]:
vessel_class_metrics = api.get_metrics_by_vessel_class_id(vessel_class_id=86)
# vessel_class_metrics

### Get Metrics for a vessel class for a specific year

In [14]:
vessel_class_metrics = api.get_metrics_by_vessel_class_id(vessel_class_id=86, year=2020)
# vessel_class_metrics